# Getting the coordinates from the cities
Getting the coordinates from [this online file](https://docs.google.com/spreadsheets/d/1nNLhqt2C2_BQgeez8H_-Oa-lwaHFA6sudBMy-TqXcjA/edit?gid=0#gid=0). Make sure to download the csv.
I can streamline this, maybe when I feel like it one time. 

See the README for more info. This is a link to the ma

In [1]:
import re 
import pandas as pd
import numpy as np 
from OSMPythonTools.nominatim import Nominatim

In [6]:
#NOTE: this not secure, but that is not important for when we process only this data
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [7]:
# credits to https://gis.stackexchange.com/questions/54405/is-there-a-way-to-populate-a-spreadsheet-of-city-names-with-their-latitudes-and

# Converts a iterable of string City names to a list of lat lon stings
# Example: Input - "New York, NY, USA"
#          Output = "40.7127281,-74.0060152"
def QueryLocations(city_names):
    nominatim = Nominatim()
    lats, lons = [], [] 
    for name in city_names:
        print(name)
        city_json = nominatim.query(name).toJSON()[0]

        lats.append(city_json["lat"])
        lons.append(city_json["lon"])
        
    return lats, lons

In [8]:
df = pd.read_csv("data/European AIS groups - map.csv", skiprows=1)
df = df.loc[df["status"] == "Ready to add"]

In [9]:
%%capture 
#capture because it produces much unnecessary output
lats, lons = QueryLocations(df['City'])

In [10]:
# have to use these specific column names for https://maps.co/gis/
df['lat'] = lats
df['lng'] = lons

df["lng"] = df["lng"].astype(float)
df["lat"] = df["lat"].astype(float)

In [11]:
# df.dtypes

In [12]:
# manually fix some coords
df.loc[df['City'] == "Cracow", 'lat'] = 50.0647
df.loc[df['City'] == "Cracow", 'lng'] = 19.9450

df.loc[df['City'] == "Belgrade", 'lat'] = 44.8125
df.loc[df['City'] == "Belgrade", 'lng'] = 20.4612


In [14]:
# df[["group_type", "lat", "lng"]]

def adjust_longitude_if_overlap(df):
    something_changed = False
    df_orig = df.copy()
    for i in range(len(df_orig)):
        for j in range(i+1, len(df_orig)):
            if df.loc[i, 'lat'] == df.loc[j, 'lat'] and df.loc[i, 'lng'] == df.loc[j, 'lng']:
                # Adjust longitude by roughly 2 km, assuming 1 degree of longitude is roughly 111 km
                adjustment = 2 / 111
                df.at[j, 'lng'] += adjustment
                something_changed = True

    return df, something_changed

# Apply the adjustment to the DataFrame
while True:
    df, something_changed = adjust_longitude_if_overlap(df)
    if not something_changed:
        break

# df

In [15]:
# df[["group_type", "lat", "lng"]]

def adjust_longitude_if_overlap(df):
    something_changed = False
    df_orig = df.copy()
    for i in range(len(df_orig)):
        for j in range(i+1, len(df_orig)):
            if df.loc[i, 'lat'] == df.loc[j, 'lat'] and df.loc[i, 'lng'] == df.loc[j, 'lng']:
                # Adjust longitude by roughly 2 km, assuming 1 degree of longitude is roughly 111 km
                adjustment = 2 / 111
                df.at[j, 'lng'] += adjustment
                something_changed = True

    return df, something_changed

# Apply the adjustment to the DataFrame
while True:
    df, something_changed = adjust_longitude_if_overlap(df)
    if not something_changed:
        break

# df

In [16]:
df = df.fillna('')

df[["orgname", "website / email (for contact persons)", "last updated"]].astype("string")

# ugly code but oh well 
# combine the different colums to create the web representation
df["text"] = df.apply(lambda row: re.sub(r'\n{3,}', '\n\n', f"{row['orgname']}\n{row['website / email (for contact persons)']}\n\n{row['other_public']}\n\nLast updated: {row['last updated']}").rstrip('\n'), axis=1)
# df["description"] = df.apply(lambda row: re.sub(r'\n{3,}', '\n\n', f"{row['orgname']}\n{row['website / email (for contact persons)']}\n\n{row['other_public']}\n\n{row['last updated']}").rstrip('\n'), axis=1)


In [17]:
group_types = ["university", "city", "national", "ais co-working space", "contact person"]

In [18]:
df["group_type"] = df["group_type"].str.lower().str.strip()
for i in df["group_type"]:
    if i not in group_types:
        raise ValueError(f"Group type is wrong for {i}")

In [19]:
# def adjust_longitude_if_overlap(df):
#     for i in range(len(df)):
#         for j in range(i+1, len(df)):
#             if df.loc[i, 'lat'] == df.loc[j, 'lat'] and df.loc[i, 'lng'] == df.loc[j, 'lng']:
#                 # Adjust longitude by roughly 2 km, assuming 1 degree of longitude is roughly 111 km
#                 adjustment = 2 / 111
#                 print(j)
#                 df.at[j, 'lng'] += adjustment

#     return df

# # Apply the adjustment to the DataFrame
# df = adjust_longitude_if_overlap(df)

In [20]:
for group_type in group_types:
    category_df = df[["lat", "lng", "text"]].loc[df['group_type']==group_type].to_csv(f"data/{group_type}.csv")


## Good, you have the files
Now: login here: https://umap.openstreetmap.fr/en/me

You see the map. 

Upload the files to the relevant layers. 